<a href="https://colab.research.google.com/github/fopamesmin/movie-project/blob/main/laliga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install colorama

In [7]:
import os
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as BS
import logging

# Constants
sofifa_url = "https://www.laliga.com/en-ES/laliga-easports/standing"
COLUMNS = ["position", "team", "Points", "w", "D", "L", "GF", "GA", "GD"]
ID_seasonId = ["2022/2023"]

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}

logging.basicConfig(level=logging.INFO)
dataset_url = "./DataSet"
# Ensure the directory exists
os.makedirs(dataset_url, exist_ok=True)

for season in ID_seasonId:
    teams_list = []

    response = req.get(sofifa_url, headers=headers)

    if response.status_code == req.codes.ok:
        soup = BS(response.content, "lxml")

        # Find the table rows containing the standings
        rows = soup.select("div.styled__StandingTabBody-sc-e89col-0.isRHqh")

        for row in rows:
            team_data = []
            try:
                # Extract data using the appropriate classes
                position = row.find("div", class_="styled__Td-sc-e89col-10 fTFWtb").find("p").text.strip()
                team = row.find("div", class_="styled__ShieldContainer-sc-1opls7r-0 eIaTDi shield-desktop").find("p").text.strip()
                points = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[0].find("p").text.strip()
                w = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[1].find("p").text.strip()
                d = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[2].find("p").text.strip()
                l = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[3].find("p").text.strip()
                gf = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[4].find("p").text.strip()
                ga = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[5].find("p").text.strip()
                gd = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[6].find("p").text.strip()

                team_data.extend([position, team, points, w, d, l, gf, ga, gd])
                teams_list.append(team_data)
            except Exception as e:
                logging.error(f"Error parsing row: {e}")
                continue

    else:
        logging.error(f"Failed to retrieve data for season {season}")

    df = pd.DataFrame(teams_list, columns=COLUMNS)

    csv_file_path = os.path.join(dataset_url, f"{season.replace('/', '-')}.csv")
    df.to_csv(csv_file_path, header=True, index=False)

    print(f"Data for season {season}:\n", df)

Data for season 2022/2023:
    position                team Points   w   D   L  GF  GA  GD
0         1         Real Madrid     94  37  29   7   1  87  26
1         2        FC Barcelona     82  37  25   7   5  77  43
2         3           Girona FC     81  38  25   6   7  85  46
3         4  Atlético de Madrid     73  37  23   4  10  68  43
4         5       Athletic Club     65  37  18  11   8  60  37
5         6       Real Sociedad     60  37  16  12   9  51  37
6         7          Real Betis     56  37  14  14   9  48  45
7         8       Villarreal CF     52  37  14  10  13  64  64
8         9         Valencia CF     48  37  13   9  15  38  43
9        10    Deportivo Alavés     45  37  12   9  16  35  45
10       11          CA Osasuna     44  37  12   8  17  44  55
11       12           Getafe CF     43  37  10  13  14  41  52
12       13          Sevilla FC     41  37  10  11  16  47  52
13       14            RC Celta     40  37  10  10  17  44  55
14       15       UD Las Pa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_url = "/content/drive/MyDrive/DataSet"